In [8]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,8)

In [9]:
# Import pipeline modules
from pipeline.extract import load_traffy_data, load_single_location_weather, load_bangkok_air_quality
from pipeline.utils import split_coordinates, clean_traffy_data
from pipeline.preprocess import parse_type_column, filter_empty_types, drop_missing_weather

In [10]:
# Load and clean Traffy data using pipeline
df_traffy = load_traffy_data('data/raw/bangkok_traffy.csv')
df_traffy = clean_traffy_data(df_traffy)
df_traffy = split_coordinates(df_traffy)

print(f"Cleaned Traffy data: {df_traffy.shape}")
df_traffy.head()

Loading Traffy data from: data/raw/bangkok_traffy.csv
✓ Loaded 787,026 records
✓ Loaded 787,026 records
Cleaned Traffy data: (651600, 9)
Cleaned Traffy data: (651600, 9)


Loading Traffy data from: data/raw/bangkok_traffy.csv
✓ Loaded 787,026 records
✓ Loaded 787,026 records
Cleaned Traffy data: (651600, 9)
Cleaned Traffy data: (651600, 9)


,type,comment,coords,subdistrict,district,province,timestamp,longitude,latitude
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,100.66709,13.67891
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,100.52649,13.72060
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,100.59165,13.82280
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,100.59131,13.80910
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,100.50848,13.77832


In [11]:
# Load weather data
df_weather = load_single_location_weather('data/raw/open-meteo-13.74N100.50E9m.csv')
print(f"Weather data: {df_weather.shape}")
df_weather.head()

Weather data: (30696, 10)


,time,temperature_2m (°C),dew_point_2m (°C),relative_humidity_2m (%),rain (mm),vapour_pressure_deficit (kPa),cloud_cover (%),wind_direction_10m (°),surface_pressure (hPa),wind_speed_10m (km/h)
0,2021-08-01T00:00,27.2,24.4,84,0.0,0.56,65,215,1006.1,5.7
1,2021-08-01T01:00,28.8,24.5,78,0.0,0.89,90,207,1006.8,9.7
2,2021-08-01T02:00,29.5,23.9,72,0.1,1.16,99,235,1006.9,13.2
3,2021-08-01T03:00,29.9,23.9,70,0.0,1.26,97,249,1006.6,15.8
4,2021-08-01T04:00,31.0,23.9,66,0.1,1.52,95,254,1005.9,16.5


In [12]:
weather_df = pd.read_csv('data/raw/open-meteo-13.74N100.50E9m.csv')
# D:\CEDT\02_01\data_Sci\dsde-final\data\raw\open-meteo-13.74N100.50E9m.csv
weather_df.info()

✓ Timestamps parsed and date columns created


In [13]:
traffy = df.drop(['ticket_id', 'organization', 'photo', 'photo_after',
            'address', 'state', 'star', 'count_reopen', 'last_activity'], axis=1)

traffy.dropna(inplace=True)

traffy = traffy[traffy['province'] == 'กรุงเทพมหานคร'].reset_index(drop=True)

✓ Daily weather aggregated: (1279, 10)


,date,temperature_2m (°C),dew_point_2m (°C),relative_humidity_2m (%),rain (mm),vapour_pressure_deficit (kPa),cloud_cover (%),wind_direction_10m (°),surface_pressure (hPa),wind_speed_10m (km/h)
0,2021-08-01,28.566667,23.866667,76.166667,0.133333,0.971667,96.666667,242.250000,1005.037500,11.608333
1,2021-08-02,28.408333,23.991667,77.541667,0.045833,0.906667,94.291667,225.666667,1005.466667,11.212500
2,2021-08-03,27.979167,23.975000,79.375000,0.245833,0.815417,76.375000,230.666667,1006.320833,10.858333
3,2021-08-04,28.162500,24.233333,79.583333,0.120833,0.807917,86.625000,215.416667,1006.133333,10.862500
4,2021-08-05,28.175000,24.304167,80.041667,0.204167,0.793750,92.041667,210.875000,1006.183333,10.554167


✓ Loaded 4,351 air quality records


,date,pm25,pm10,o3,no2
0,2025-12-01,136,60,30,17
1,2025-12-02,137,56,33,15
2,2025-12-03,127,37,19,12
3,2025-12-04,85,29,13,7
4,2025-12-05,68,38,1,16


In [15]:
traffy.info()
traffy['province'].mode()

✓ Merged with weather: (651600, 19)
✓ Merged with air quality: (651600, 23)
✓ Merged with air quality: (651600, 23)


✓ Merged with weather: (651600, 19)
✓ Merged with air quality: (651600, 23)
✓ Merged with air quality: (651600, 23)


,type,comment,coords,subdistrict,district,province,timestamp,longitude,latitude,date,...,rain (mm),vapour_pressure_deficit (kPa),cloud_cover (%),wind_direction_10m (°),surface_pressure (hPa),wind_speed_10m (km/h),pm25,pm10,o3,no2
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00:00,100.66709,13.67891,2021-09-19,...,0.091667,0.559167,96.000000,216.500000,1007.729167,5.462500,52,27,9,4
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00:00,100.52649,13.72060,2021-09-26,...,1.575000,0.223750,99.750000,132.333333,1006.575000,8.158333,25,18,8,6
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00:00,100.59165,13.82280,2021-12-09,...,0.000000,1.338333,78.250000,63.833333,1013.483333,8.387500,100,54,20,15
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00:00,100.59131,13.80910,2021-12-13,...,0.000000,1.427083,21.333333,68.375000,1012.020833,10.070833,79,65,23,14
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00:00,100.50848,13.77832,2021-12-17,...,0.000000,1.851250,99.833333,138.541667,1009.895833,9.745833,117,48,18,6


In [16]:
traffy.head()

✓ After preprocessing: (540633, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540633 entries, 0 to 540632
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype              
---  ------                         --------------   -----              
 0   type                           540633 non-null  object             
 1   comment                        540633 non-null  object             
 2   coords                         540633 non-null  object             
 3   subdistrict                    540633 non-null  object             
 4   district                       540633 non-null  object             
 5   province                       540633 non-null  object             
 6   timestamp                      540633 non-null  datetime64[ns, UTC]
 7   longitude                      540633 non-null  float64            
 8   latitude                       540633 non-null  float64            
 9   date                           540633 non-nul

In [17]:
# Save the cleaned `traffy` DataFrame to disk
import os
os.makedirs('data/processed', exist_ok=True)
traffy.to_csv('data/processed/traffy_clean.csv', index=False)
print('Saved: data/processed/traffy_clean.csv')

✓ Saved: data/processed/traffy_weather_merged.csv


In [ ]:
clean_df = pd.read_csv('data/processed/traffy_clean.csv')

In [ ]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   type         651600 non-null  object
 1   comment      651600 non-null  object
 2   coords       651600 non-null  object
 3   subdistrict  651600 non-null  object
 4   district     651600 non-null  object
 5   province     651600 non-null  object
 6   timestamp    651600 non-null  object
dtypes: object(7)
memory usage: 34.8+ MB


In [ ]:
clean_df.head()

,type,comment,coords,subdistrict,district,province,timestamp
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00


In [21]:
air_df = pd.read_csv('data/processed/bangkok-air-quality.csv')
air_df.head()

,date,pm25,pm10,o3,no2,so2,co
0,2025/12/1,136,60,30,17,,
1,2025/12/2,137,56,33,15,,
2,2025/12/3,127,37,19,12,,
3,2025/12/4,85,29,13,7,,
4,2025/12/5,68,38,1,16,,


In [22]:
print(air_df.columns.tolist())

['date', ' pm25', ' pm10', ' o3', ' no2', ' so2', ' co']


In [23]:
air_df = air_df.drop([' so2',' co'], axis=1)

In [24]:
air_df.head()

,date,pm25,pm10,o3,no2
0,2025/12/1,136,60,30,17
1,2025/12/2,137,56,33,15
2,2025/12/3,127,37,19,12
3,2025/12/4,85,29,13,7
4,2025/12/5,68,38,1,16


In [25]:
# clean_df['date'] = pd.to_datetime(clean_df['timestamp']).dt.strftime('%Y-%m-%d')
clean_df['date'] = pd.to_datetime(clean_df['timestamp'], format='ISO8601').dt.date


In [26]:
clean_df['date'] = pd.to_datetime(clean_df['date']).dt.normalize()

In [27]:
clean_df.info()
clean_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   type         651600 non-null  object        
 1   comment      651600 non-null  object        
 2   coords       651600 non-null  object        
 3   subdistrict  651600 non-null  object        
 4   district     651600 non-null  object        
 5   province     651600 non-null  object        
 6   timestamp    651600 non-null  object        
 7   date         651600 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 39.8+ MB


,type,comment,coords,subdistrict,district,province,timestamp,date
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,2021-09-19
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,2021-09-26
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,2021-12-09
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,2021-12-13
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,2021-12-17


In [28]:
air_df['date'] = pd.to_datetime(air_df['date']).dt.normalize()

In [29]:
air_df.info()
air_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4351 entries, 0 to 4350
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4351 non-null   datetime64[ns]
 1    pm25   4351 non-null   object        
 2    pm10   4351 non-null   object        
 3    o3     4351 non-null   object        
 4    no2    4351 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 170.1+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4351 entries, 0 to 4350
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4351 non-null   datetime64[ns]
 1    pm25   4351 non-null   object        
 2    pm10   4351 non-null   object        
 3    o3     4351 non-null   object        
 4    no2    4351 non-null   object        
dtypes: datetime64[ns](1), object(4)
memory usage: 170.1+ KB


,date,pm25,pm10,o3,no2
0,2025-12-01,136,60,30,17
1,2025-12-02,137,56,33,15
2,2025-12-03,127,37,19,12
3,2025-12-04,85,29,13,7
4,2025-12-05,68,38,1,16


In [30]:
merged_df = clean_df.merge(air_df, on='date', how='left')

In [31]:
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651600 entries, 0 to 651599
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   type         651600 non-null  object        
 1   comment      651600 non-null  object        
 2   coords       651600 non-null  object        
 3   subdistrict  651600 non-null  object        
 4   district     651600 non-null  object        
 5   province     651600 non-null  object        
 6   timestamp    651600 non-null  object        
 7   date         651600 non-null  datetime64[ns]
 8    pm25        651138 non-null  object        
 9    pm10        651138 non-null  object        
 10   o3          651138 non-null  object        
 11   no2         651138 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 59.7+ MB


,type,comment,coords,subdistrict,district,province,timestamp,date,pm25,pm10,o3,no2
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,2021-09-19,52,27,9,4
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,2021-09-26,25,18,8,6
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,2021-12-09,100,54,20,15
3,{},หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,2021-12-13,79,65,23,14
4,{},ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,2021-12-17,117,48,18,6


In [35]:
merged_df['type'] = merged_df['type'].replace("{}", "ไม่ระบุ")
import ast

def to_list(x):
    if x == "ไม่ระบุ" or x == "{}":
        return []
    # แปลง "{น้ำท่วม,ถนน}" → ["น้ำท่วม","ถนน"]
    return [i.strip() for i in x.strip("{}").split(",")]

merged_df['type_list'] = merged_df['type'].apply(to_list)

os.makedirs('data/processed', exist_ok=True)
merged_df.to_csv('data/processed/traffy_merged.csv', index=False)
print('Saved: data/processed/traffy_merged.csv')

C:\Users\grain\AppData\Local\Temp\ipykernel_6488\1912563364.py:2: DtypeWarning: Columns (19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('data/processed/traffy_weather_merged.csv')


Final dataset shape: (540633, 23)

Column names: ['type', 'comment', 'coords', 'subdistrict', 'district', 'province', 'timestamp', 'longitude', 'latitude', 'date', 'temperature_2m (°C)', 'dew_point_2m (°C)', 'relative_humidity_2m (%)', 'rain (mm)', 'vapour_pressure_deficit (kPa)', 'cloud_cover (%)', 'wind_direction_10m (°)', 'surface_pressure (hPa)', 'wind_speed_10m (km/h)', ' pm25', ' pm10', ' o3', ' no2']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540633 entries, 0 to 540632
Data columns (total 23 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   type                           540633 non-null  object 
 1   comment                        540633 non-null  object 
 2   coords                         540633 non-null  object 
 3   subdistrict                    540633 non-null  object 
 4   district                       540633 non-null  object 
 5   province                       540633 non-null

In [33]:
df_t= pd.read_csv('data/processed/traffy_merged.csv')
# df_t.shape
df_t.head()

C:\Users\grain\AppData\Local\Temp\ipykernel_6488\2761252754.py:1: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_t= pd.read_csv('data/processed/traffy_merged.csv')


,type,comment,coords,subdistrict,district,province,timestamp,date,pm25,pm10,o3,no2,type_list
0,"{น้ำท่วม,ร้องเรียน}",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,2021-09-19,52,27,9,4,"['น้ำท่วม', 'ร้องเรียน']"
1,{สะพาน},สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,2021-09-26,25,18,8,6,['สะพาน']
2,"{น้ำท่วม,ถนน}",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,2021-12-09,100,54,20,15,"['น้ำท่วม', 'ถนน']"
3,ไม่ระบุ,หน้าปากซอย ลาดพร้าววังหิน26,"100.59131,13.80910",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,2021-12-13,79,65,23,14,[]
4,ไม่ระบุ,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,"100.50848,13.77832",ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,2021-12-17,117,48,18,6,[]
